In [20]:
import awkward as ak
import numpy as np
events = ak.from_parquet("./output/example_photons.parquet")

ValueError: while calling

    ak.from_parquet(
        path = './output/example_photons.parquet'
        columns = None
        row_groups = None
        storage_options = None
        max_gap = 64000
        max_block = 256000000
        footer_sample_size = 1000000
        generate_bitmasks = False
        highlevel = True
        behavior = None
    )

Error details: no *.parquet or *.parq matches for path './output/example_photons.parquet'

## Let's see how many events we simulated

In [6]:
print(f"Number of events simulated: {len(events)}")

Number of events simulated: 10


## We can access the true Monte Carlo information by looking in the `mc_truth` field

Let's do this now for the a single event. Note that in what follows, you can replace `event` with `events` to get the information for all events

In [7]:
event = events[7]
event["mc_truth"]

<Record {interaction: 1, ...} type='{interaction: int64, initial_state_ener...'>

In [8]:
# Convenience dictionary for reading the integer particle encoding
pdg_dict = {
    11: "E-",
    12: "NuE",
    13: "Mu-",
    14: "NuMu",
    15: "Tau-",
    16: "NuTau",
    -11: "E+",
    -12: "NuEBar",
    -13: "Mu+",
    -14: "NuMuBar",
    -15: "Tau+",
    -16: "NuTauBar",
    -2000001006: "Hadrons"
}
# Convenience dictionary for reading interaction type
interaction_dict = {
    0: "Neutral current",
    1: "Charged current",
    2: "Glashow resonance"
}

## The properties of the initial state have the form `initial_state_*`.

Let's take a peak at some of these for our event !

In [9]:
f"We can look at {[x for x in event['mc_truth'].fields if 'initial' in x]}"

"We can look at ['initial_state_energy', 'initial_state_type', 'initial_state_zenith', 'initial_state_azimuth', 'initial_state_x', 'initial_state_y', 'initial_state_z']"

In [10]:
init_type = event["mc_truth", "initial_state_type"]
init_e = event["mc_truth", "initial_state_energy"]
print(f"This initial particle was a {pdg_dict[init_type]} with energy {init_e} GeV.")

This initial particle was a NuE with energy 1080.9425009611589 GeV.


In [11]:
init_vertex = np.array([
    event["mc_truth", "initial_state_x"],
    event["mc_truth", "initial_state_y"],
    event["mc_truth", "initial_state_z"]
])
print(f"The interaction vertex was at {init_vertex} m")

The interaction vertex was at [  271.38172879   554.00280532 -2217.92241714] m


## Enough of that, let's look at the final state information

In [12]:
f"We can look at {[x for x in event['mc_truth'].fields if 'final' in x]}"

"We can look at ['final_state_energy', 'final_state_type', 'final_state_zenith', 'final_state_azimuth', 'final_state_x', 'final_state_y', 'final_state_z', 'final_state_parent']"

Since we can't know *a priori* how many final states there will be---think $\tau$ decay and such---these must be stored in a list. Let's see what type of particles these were. We can also see if this matches with the expected interaction type.

In [13]:
final_type = event["mc_truth", "final_state_type"]
print(f"The final products of this interaction are {[pdg_dict[x] for x in final_type]}")

interaction_type = event["mc_truth", "interaction"]
print(f"This was a {interaction_dict[interaction_type].lower()} interaction.")

The final products of this interaction are ['E-', 'Hadrons']
This was a charged current interaction.


In [14]:
final_e = event["mc_truth", "final_state_energy"]
print(f"The final particles had energies {final_e} GeV")

The final particles had energies [501, 580] GeV


## Now we'll check out the photons that arrived at the detector.

Let's see what fields are available to us.

In [15]:
event["photons"].fields

['sensor_pos_x',
 'sensor_pos_y',
 'sensor_pos_z',
 'string_id',
 'sensor_id',
 't',
 'id_idx']

We can find the number of photons that reached an OM by checking the length of any of the array that are stored in these fields

In [16]:
print(f"The first event produced {len(event['photons', 't'])} photons that reached an OM")

unique_om = list(set(x for x in zip(event["photons", "string_id"], event["photons", "sensor_id"])))
print(f"The number of OMs that saw light is {len(unique_om)}")

The first event produced 21 photons that reached an OM
The number of OMs that saw light is 17


In [17]:
times = event["photons", "t"]
print(f"The first photon arrived at {np.min(times)} ns and the last one arrived at {np.max(times)} ns")

The first photon arrived at 732.696655 ns and the last one arrived at 2691.547363 ns


Finally, although it is a bit convoluted, we are able to find which of the final states produced each photon.

In [18]:
which_final = event["mc_truth", "final_state_type", event["photons", "id_idx"]-1]
print([pdg_dict[x] for x in which_final])

['E-', 'E-', 'E-', 'E-', 'E-', 'E-', 'E-', 'E-', 'E-', 'E-', 'E-', 'E-', 'E-', 'E-', 'Hadrons', 'Hadrons', 'Hadrons', 'Hadrons', 'Hadrons', 'Hadrons', 'Hadrons']


## Finally, let's get the `Prometheus` configuration information out of this file

In [19]:
import pyarrow.parquet as pq
import json

config = json.loads(pq.read_metadata('./output/example_photons.parquet').metadata[b'config_prometheus'])
for k, v in config.items():
    print(k)
    print(v)
    print()

general
{'version': 'github'}

run
{'run number': 853, 'nevents': 10, 'storage prefix': './output/', 'outfile': './output/EMinus_Hadrons_seed_853_photons.parquet', 'random state seed': 853, 'subset': None, 'meta_name': 'meta_data_853'}

detector
{'geo file': '../resources/geofiles/icecube.geo', 'padding': 200}

injection
{'name': 'LeptonInjector', 'LeptonInjector': {'inject': True, 'paths': {'install location': '/n/holylfs05/LABS/arguelles_delgado_lab/Lab/common_software//lib64/', 'xsec dir': '../resources//cross_section_splines/', 'earth model location': '/n/home12/jlazar/prometheus/resources/earthparams/densities/PREM_south_pole.dat', 'injection file': './output/EMinus_Hadrons_seed_853_LI_output.h5', 'lic file': '/n/home12/jlazar/prometheus/examples/output/853_LI_config.lic', 'diff xsec': '/n/home12/jlazar/prometheus/resources/cross_section_splines/dsdxdy_nubar_CC_iso.fits', 'total xsec': '/n/home12/jlazar/prometheus/resources/cross_section_splines/sigma_nubar_CC_iso.fits', 'lic name